# Create the model

In [11]:
## import necesarry libraries
import pandas as pd

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier

In [12]:
## read in data
## read in data
instances_long = pd.read_csv('data/ships_extended.csv', index_col=0)
instances_short = pd.read_csv('data/west_extended.csv')

X_train, X_test, Y_train, Y_test = train_test_split(instances_long.drop(['epithet_gr', 'epithet_en', 'clause', 'bigrams', 'trigrams'], axis=1), instances_long['epithet_en'], test_size=0.2)

In [13]:
## BASELINE ONE_R MODEL
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2)
one_r = DecisionTreeClassifier(max_depth=1)
one_r = one_r.fit(x_train, y_train)
cross_val_score(one_r, x_test, y_test, scoring='f1_micro').mean()

/home/jwilliamsmurnane/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.5800000000000001

In [14]:
## DECISION TREE
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2)
one_r = DecisionTreeClassifier(max_depth=5)
one_r = one_r.fit(x_train, y_train)
cross_val_score(one_r, x_test, y_test, scoring='f1_micro').mean()

/home/jwilliamsmurnane/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.6933333333333335